In [5]:
import numpy as np
import os
import shutil
import subprocess



In [6]:
total_count = 48


replica_count = 4

n_clusters = int(total_count / replica_count)

nclients = n_clusters

In [7]:
regions = [ 'us-west1-b','us-west1-b','us-west1-b','us-west1-b', \
           'europe-west3-c', 'europe-west3-c', 'europe-west3-c', 'europe-west3-c', \
           'asia-south1-c', 'asia-south1-c', 'asia-south1-c', 'asia-south1-c']

n_regions = len(regions)

In [8]:
for i,k in enumerate(regions):
    print(i,k)

0 us-west1-b
1 us-west1-b
2 us-west1-b
3 us-west1-b
4 europe-west3-c
5 europe-west3-c
6 europe-west3-c
7 europe-west3-c
8 asia-south1-c
9 asia-south1-c
10 asia-south1-c
11 asia-south1-c


In [9]:
lines  = []
zones = {}


lim = 0
for reg in regions:
    for i in range(replica_count+int(nclients/n_clusters)):
        line = 'gcloud compute instances create resdb-'+str(lim+i)+' --project=ucr-ursa-major-lesani-lab --zone='+reg+' --machine-type=e2-small --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
        
        print(line)
        lines.append(line)
        zones[i] = reg
    
    lim = lim + replica_count+int(nclients/n_clusters)
        
    
    
    
with open('launch_cloud_instances.sh', 'w') as f:
    for item in lines:
        f.write(str(item) + '\n')


gcloud compute instances create resdb-0 --project=ucr-ursa-major-lesani-lab --zone=us-west1-b --machine-type=e2-small --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-e

In [10]:
import subprocess
import concurrent.futures

def run_command(command):
    subprocess.call(command, shell=True)

commands = lines

# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=len(commands)) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-29].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-24].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230918'.



NAME      ZONE            MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP    EXTERNAL_IP    STATUS
resdb-29  europe-west3-c  e2-small                   10.156.15.201  35.234.78.150  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-35].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-28].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-38].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230918'.



NAME      ZONE            MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP    EXTERNAL_IP    STATUS
resdb-24  europe-west3-c  e2-small                   10.156.15.200  34.141.33.191  RUNNING


 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230918'.

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-22].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230918'.

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-37].


...

NAME      ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP    EXTERNAL_IP    STATUS
resdb-47  asia-south1-c  e2-small                   10.160.15.202  34.93.234.183  RUNNING
NAME      ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP    EXTERNAL_IP    STATUS
resdb-52  asia-south1-c  e2-small                   10.160.15.211  34.93.176.138  RUNNING


 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230918'.



NAME      ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP    EXTERNAL_IP    STATUS
resdb-53  asia-south1-c  e2-small                   10.160.15.213  34.100.206.58  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/asia-south1-c/instances/resdb-58].
 - The resource 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831' is deprecated. A suggested replacement is 'projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230918'.



NAME      ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP    EXTERNAL_IP   STATUS
resdb-58  asia-south1-c  e2-small                   10.160.15.210  34.93.95.143  RUNNING


In [6]:
# os.system('sh launch_cloud_instances.sh')

In [11]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [12]:
# os.system("aws --region ap-south-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

# os.system("aws --region ap-south-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


instance_type = 'e2-small'


for index, region  in enumerate(regions[::4]):
    
    print(index, region)
    redirect = '>>'
    if index==0:
        redirect = '>'
        
    os.system('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
              redirect+' all_internal_ips')
    
    
    print('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
              redirect+' all_internal_ips')


    os.system('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_external_ips')
    
    
    print('gcloud compute instances list --filter="ZONE='+region+'" --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_external_ips')
    
    










0 us-west1-b
gcloud compute instances list --filter="ZONE=us-west1-b" --format="value(networkInterfaces[0].networkIP)" > all_internal_ips
gcloud compute instances list --filter="ZONE=us-west1-b" --format="value(networkInterfaces[0].networkIP)" > all_external_ips
1 europe-west3-c
gcloud compute instances list --filter="ZONE=europe-west3-c" --format="value(networkInterfaces[0].networkIP)" >> all_internal_ips
gcloud compute instances list --filter="ZONE=europe-west3-c" --format="value(networkInterfaces[0].networkIP)" >> all_external_ips
2 asia-south1-c
gcloud compute instances list --filter="ZONE=asia-south1-c" --format="value(networkInterfaces[0].networkIP)" >> all_internal_ips
gcloud compute instances list --filter="ZONE=asia-south1-c" --format="value(networkInterfaces[0].networkIP)" >> all_external_ips


In [13]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [14]:
ipList, len(ipList)

(['10.138.15.215',
  '10.138.15.221',
  '10.138.15.226',
  '10.138.15.220',
  '10.138.15.228',
  '10.138.15.227',
  '10.138.15.225',
  '10.138.15.214',
  '10.138.15.217',
  '10.138.15.219',
  '10.138.15.212',
  '10.138.15.222',
  '10.138.15.223',
  '10.138.15.229',
  '10.138.15.218',
  '10.138.15.211',
  '10.138.15.213',
  '10.138.15.224',
  '10.138.15.210',
  '10.138.15.216',
  '10.156.15.219',
  '10.156.15.211',
  '10.156.15.214',
  '10.156.15.202',
  '10.156.15.200',
  '10.156.15.215',
  '10.156.15.216',
  '10.156.15.205',
  '10.156.15.207',
  '10.156.15.201',
  '10.156.15.210',
  '10.156.15.203',
  '10.156.15.206',
  '10.156.15.213',
  '10.156.15.204',
  '10.156.15.208',
  '10.156.15.218',
  '10.156.15.217',
  '10.156.15.209',
  '10.156.15.212',
  '10.160.15.196',
  '10.160.15.205',
  '10.160.15.203',
  '10.160.15.208',
  '10.160.15.197',
  '10.160.15.199',
  '10.160.15.206',
  '10.160.15.202',
  '10.160.15.204',
  '10.160.15.195',
  '10.160.15.209',
  '10.160.15.207',
  '10.160.15

In [15]:
clusters_per_region_map = {}

clusters_per_region_map[0] = 0
clusters_per_region_map[1] = 0
clusters_per_region_map[2] = 0

n_regions = 3

for i in range(n_clusters):
    
    
    clusters_per_region_map[i%n_regions] = clusters_per_region_map[i%n_regions] + 1

In [16]:
clusters_per_region_map

{0: 4, 1: 4, 2: 4}

In [17]:
new_server_only_ips= []
new_client_only_ips= []


current_region_total = 0
for i in range(n_regions):
    new_server_only_ips = new_server_only_ips + ipList[current_region_total:current_region_total+ clusters_per_region_map[i] *replica_count]
    
    new_client_only_ips = new_client_only_ips + ipList[current_region_total+ clusters_per_region_map[i] *replica_count:
                                                      current_region_total+ clusters_per_region_map[i] *(replica_count+1)]
    
    
    current_region_total = current_region_total + clusters_per_region_map[i] *replica_count +  clusters_per_region_map[i]
    
    

In [18]:
new_server_only_ips, len(new_server_only_ips)

(['10.138.15.215',
  '10.138.15.221',
  '10.138.15.226',
  '10.138.15.220',
  '10.138.15.228',
  '10.138.15.227',
  '10.138.15.225',
  '10.138.15.214',
  '10.138.15.217',
  '10.138.15.219',
  '10.138.15.212',
  '10.138.15.222',
  '10.138.15.223',
  '10.138.15.229',
  '10.138.15.218',
  '10.138.15.211',
  '10.156.15.219',
  '10.156.15.211',
  '10.156.15.214',
  '10.156.15.202',
  '10.156.15.200',
  '10.156.15.215',
  '10.156.15.216',
  '10.156.15.205',
  '10.156.15.207',
  '10.156.15.201',
  '10.156.15.210',
  '10.156.15.203',
  '10.156.15.206',
  '10.156.15.213',
  '10.156.15.204',
  '10.156.15.208',
  '10.160.15.196',
  '10.160.15.205',
  '10.160.15.203',
  '10.160.15.208',
  '10.160.15.197',
  '10.160.15.199',
  '10.160.15.206',
  '10.160.15.202',
  '10.160.15.204',
  '10.160.15.195',
  '10.160.15.209',
  '10.160.15.207',
  '10.160.15.211',
  '10.160.15.213',
  '10.160.15.200',
  '10.160.15.198'],
 48)

In [21]:
new_client_only_ips, len(new_client_only_ips) 

(['10.138.15.213',
  '10.138.15.224',
  '10.138.15.210',
  '10.138.15.216',
  '10.156.15.218',
  '10.156.15.217',
  '10.156.15.209',
  '10.156.15.212',
  '10.160.15.214',
  '10.160.15.201',
  '10.160.15.210',
  '10.160.15.212'],
 12)

In [22]:
ipList = new_server_only_ips + new_client_only_ips

In [23]:
ipList, len(ipList)

(['10.138.15.215',
  '10.138.15.221',
  '10.138.15.226',
  '10.138.15.220',
  '10.138.15.228',
  '10.138.15.227',
  '10.138.15.225',
  '10.138.15.214',
  '10.138.15.217',
  '10.138.15.219',
  '10.138.15.212',
  '10.138.15.222',
  '10.138.15.223',
  '10.138.15.229',
  '10.138.15.218',
  '10.138.15.211',
  '10.156.15.219',
  '10.156.15.211',
  '10.156.15.214',
  '10.156.15.202',
  '10.156.15.200',
  '10.156.15.215',
  '10.156.15.216',
  '10.156.15.205',
  '10.156.15.207',
  '10.156.15.201',
  '10.156.15.210',
  '10.156.15.203',
  '10.156.15.206',
  '10.156.15.213',
  '10.156.15.204',
  '10.156.15.208',
  '10.160.15.196',
  '10.160.15.205',
  '10.160.15.203',
  '10.160.15.208',
  '10.160.15.197',
  '10.160.15.199',
  '10.160.15.206',
  '10.160.15.202',
  '10.160.15.204',
  '10.160.15.195',
  '10.160.15.209',
  '10.160.15.207',
  '10.160.15.211',
  '10.160.15.213',
  '10.160.15.200',
  '10.160.15.198',
  '10.138.15.213',
  '10.138.15.224',
  '10.138.15.210',
  '10.138.15.216',
  '10.156.15

In [24]:
clusterIpMap = {}

for i in range(len(ipList)):
    clusterIpMap[i] = ipList[i]

    
for i in range(nclients):
    
    clusterIpMap[7001+i] = ipList[-(nclients-i)]




clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(10000)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(20000)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(10000)
    clusterPortMap2[7001+i] = str(20000)



In [25]:
clusterPortMap1, clusterPortMap2

({0: '10000',
  1: '10000',
  2: '10000',
  3: '10000',
  4: '10000',
  5: '10000',
  6: '10000',
  7: '10000',
  8: '10000',
  9: '10000',
  10: '10000',
  11: '10000',
  12: '10000',
  13: '10000',
  14: '10000',
  15: '10000',
  16: '10000',
  17: '10000',
  18: '10000',
  19: '10000',
  20: '10000',
  21: '10000',
  22: '10000',
  23: '10000',
  24: '10000',
  25: '10000',
  26: '10000',
  27: '10000',
  28: '10000',
  29: '10000',
  30: '10000',
  31: '10000',
  32: '10000',
  33: '10000',
  34: '10000',
  35: '10000',
  36: '10000',
  37: '10000',
  38: '10000',
  39: '10000',
  40: '10000',
  41: '10000',
  42: '10000',
  43: '10000',
  44: '10000',
  45: '10000',
  46: '10000',
  47: '10000',
  7001: '10000',
  7002: '10000',
  7003: '10000',
  7004: '10000',
  7005: '10000',
  7006: '10000',
  7007: '10000',
  7008: '10000',
  7009: '10000',
  7010: '10000',
  7011: '10000',
  7012: '10000'},
 {0: '20000',
  1: '20000',
  2: '20000',
  3: '20000',
  4: '20000',
  5: '20000',
 

In [26]:
os.system('echo 3108 | sudo -S ./gradlew installDist')

[sudo] password for tejas: 

Starting a Gradle Daemon, 1 incompatible and 1 stopped Daemons could not be reused, use --status for details

> Task :compileJava


Note: /home/tejas/work/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.



> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

BUILD SUCCESSFUL in 15s
3 actionable tasks: 3 executed


0

In [ ]:
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

In [ ]:
os.system('echo 3108 | sudo -S ./gradlew clean')

In [ ]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

In [ ]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    
    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

In [ ]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

In [ ]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for n_cluster1 in range(n_clusters):
        
            for i_replica in range(replica_count):
    #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
                file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

        for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
            print((n_clusters*replica_count) + iter_)
            
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
            file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
        for i in range(nclients):
            file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
        file.close()


In [ ]:
replica_count

In [ ]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()
        f = int(np.ceil((replica_count-1)/3))
        print('f is',f )
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                data[n_line] = 'system.servers.num = '+str(replica_count)
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in range(replica_count):
                    data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                
                data[n_line] = data[n_line][:-1]
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

In [ ]:
for n_cluster in range(n_clusters):
    for i in range(400):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

In [ ]:
# os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

In [ ]:
os.system('git add .; git commit -m "Working with modified ClusterInfo";git push')

In [ ]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [ ]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any